In [1]:
import pandas as pd


In [2]:
df = pd.read_csv(r"C:\Users\jki\Desktop\Preactse Data sets\scanner_data.csv" )
df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [3]:
# But we’ll need to clean up the data and format the dates before we can make use of it
df.drop(['Unnamed: 0'], axis = 1)

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88
...,...,...,...,...,...,...,...
131701,04/07/2016,20203,32900,IEV,FO112,3.0,6.46
131702,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50
131703,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19
131704,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57


In [4]:
df.Date = pd.to_datetime(df.Date, format='%d/%m/%Y')

In [5]:
df.duplicated(["Date","Customer_ID"]).sum()

68979

In [7]:
df = pd.DataFrame(df.groupby(["Date","Customer_ID"]).agg({'Transaction_ID':max
                                                          , 'SKU_Category':max, 'SKU':max, 'Quantity': sum
                                                          , 'Sales_Amount':sum})).reset_index()
df.head()

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,2016-01-02,3,90,TW8,Y1M2E,4.0,10.92
1,2016-01-02,178,84,R6E,HO1M5,2.0,58.99
2,2016-01-02,195,107,LGI,VY2UB,2.0,13.10
3,2016-01-02,343,134,XG4,ZSVWE,1.0,6.75
4,2016-01-02,399,136,P42,XJLWY,2.0,10.43


In [8]:
# Assign Cohorts to Data
df['tx_month'] = df['Date'].apply(lambda x: dt.date(x.year, x.month,1))

NameError: name 'dt' is not defined

In [9]:
df['acq_month'] = df.groupby('Customer_ID')['tx_month'].transform('min')

KeyError: 'Column not found: tx_month'

In [10]:
df.loc[df['tx_month'] != df['acq_month']].head()

KeyError: 'tx_month'

In [11]:
def diff_month(x):
    d1 = x['tx_month']
    d2 = x["acq_month"]
    return ((d1.year - d2.year) * 12 + d1.month - d2.month)+1
df['cohort_idx'] = df.apply(lambda x: diff_month(x), axis=1)
df.head()

KeyError: 'tx_month'

In [12]:
def get_cohort_matrix(data, var='Customer_ID', fun=pd.Series.nunique):
    cd = data.groupby(['acq_month', 'cohort_idx'])[var].apply(fun).reset_index()
    cc = cd.pivot_table(index = 'acq_month',
                        columns = 'cohort_idx',
                        values = var)
    cs = cc.iloc[:,0]
    retention = cc.divide(cs, axis = 0)
    retention = retention.round(3) * 100
    return cc, retention

In [13]:
cc, retention = get_cohort_matrix(df)
cc

KeyError: 'acq_month'

In [14]:
# Segment Data by Quantity and Revenue
cc_sa, ret_sa = get_cohort_matrix( df, var='Sales_Amount', fun=pd.Series.median )
cc_sa

KeyError: 'acq_month'

In [15]:
# How to Forecast Cohort Data
ret_avg = [80, 70, 50, 45, 30, 25, 21,]
ret_days = [1, 2, 3, 4, 5, 6, 30]
new_users_daily = [ int(x) for x in list(df.loc[df['Date'] <= dt.datetime(2016,1,5)].groupby('Date')['Customer_ID'].apply(pd.Series.nunique).values)]
new_users_daily

NameError: name 'dt' is not defined